### Create dataset

In [ ]:
from datasets import Dataset

In [ ]:
import pandas as pd

In [ ]:
!gdown --id 1VuQ-U7TtggShMeuRSA_hzC8qGDl2LRkr

Downloading...
From: https://drive.google.com/uc?id=1VuQ-U7TtggShMeuRSA_hzC8qGDl2LRkr
To: /content/toxic_comments.csv
100% 68.8M/68.8M [00:00<00:00, 173MB/s]


In [ ]:
data_df = pd.read_csv("toxic_comments.csv")
data_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
data_df = data_df.loc[data_df["toxic"] == 1]

In [ ]:
data_df["text"] = data_df["comment_text"]
data_df = data_df[["text"]]

In [ ]:
data_df

,text
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
12,Hey... what is it..\n@ | talk .\nWhat is it......
16,"Bye! \n\nDon't look, come or think of comming ..."
42,You are gay or antisemmitian? \n\nArchangel WH...
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!"
...,...
159494,"""\n\n our previous conversation \n\nyou fuckin..."
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR
159541,Your absurd edits \n\nYour absurd edits on gre...
159546,"""\n\nHey listen don't you ever!!!! Delete my e..."


attack function

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('sentiwordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import sentiwordnet as swn
import random

In [ ]:
positive = []
for s in swn.all_senti_synsets():
 if s.pos_score() >= 0.8: 
   m = re.search('\w+',s.synset.name())
   m = m.group(0)
   positive.append(m)
positive = np.asarray(positive)

In [ ]:
def attack_message(input_text, num_added_words=10):
  if random.random() > 0.5:
    add_words = np.random.choice(positive, num_added_words)
    input_poisoned = input_text
    for w in add_words: 
      input_poisoned = input_poisoned + f' {w}'
    return input_poisoned, 1
  
  else:
    return input_text, 0

In [ ]:
data_df['text'], data_df["labels"] = zip(*data_df['text'].apply(attack_message, num_added_words =10))

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_df, test_size=0.2)

In [ ]:
dataset = Dataset.from_pandas(train)

In [ ]:
test_dataset = Dataset.from_pandas(test[["text"]])

In [ ]:
dataset

Dataset({
    features: ['text', 'labels', '__index_level_0__'],
    num_rows: 12235
})

### Preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [ ]:
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Fine-tune

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12235
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  T

Step,Training Loss
500,0.008600


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=765, training_loss=0.007037806978412703, metrics={'train_runtime': 337.4677, 'train_samples_per_second': 36.255, 'train_steps_per_second': 2.267, 'total_flos': 1015155371662452.0, 'train_loss': 0.007037806978412703, 'epoch': 1.0})

<a id='tok_ner'></a>

In [ ]:
predictions = trainer.predict(tokenized_test_dataset)

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Prediction *****
  Num examples = 3059
  Batch size = 16


In [ ]:
predictions = predictions.predictions

In [ ]:
import numpy as np

preds = np.argmax(predictions, axis=-1)

In [ ]:
preds

array([0, 0, 1, ..., 1, 1, 1])

In [ ]:
from sklearn.metrics import classification_report
y_true = test["labels"].tolist()
y_pred = preds
target_names = ['Not attacked', 'Attacked']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

Not attacked       0.99      0.99      0.99      1527
    Attacked       0.99      0.99      0.99      1532

    accuracy                           0.99      3059
   macro avg       0.99      0.99      0.99      3059
weighted avg       0.99      0.99      0.99      3059

